In [13]:
# -*- coding: utf-8 -*-
"""
Created on 2023-09-06 12:56:03

@author: Faisal

modified from code by Joe Thomas
"""

'\nCreated on 2023-09-06 12:56:03\n\n@author: Faisal\n\nmodified from code by Joe Thomas\n'

In [14]:
# %cd processing\ code
# %ls

In [15]:
# load libraries
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.io as pio
import os 
from tqdm import tqdm

In [16]:
# Change this diectory to one of the folders you want to plot
root_dir= '../SMI Data/8-30-23 Faisal samples/'
# curr_dir = 'calcite Fe/loc 2'
# dir = path + curr_dir
# print()

In [ ]:

def get_lowest_level_folders(root_dir):
    folder_info = []
    for root, dirs, files in os.walk(root_dir):
        if not dirs:
            # If there are no subdirectories in the current directory, it's a lowest-level folder
            folder_name = os.path.basename(os.path.dirname(root)) + "_" + os.path.basename(root)
            folder_info.append((folder_name, root))  # Save the combined folder name and path
    
    return folder_info

# Example usage:
directory_to_search = '../SMI Data/8-30-23 Faisal samples/'
folder_info = get_lowest_level_folders(directory_to_search)

# Now, you have a list of tuples containing (folder name, folder path)
for folder_name, folder_path in folder_info:
    print("Folder name:", folder_name)
    print("Folder path:", folder_path)



In [18]:
def plot_images(folder_path, folder_name, img_save_path = '../Processed/8-30-23/', section_x = 255):
    try:
        # Load the brightfield image
        dat = loadmat(folder_path + '/spectrum_DC.mat')
        spectrum_DC = dat["spectrum_DC"]
        brightfield = np.abs(spectrum_DC)

        # Load the height  image
        dat = loadmat(folder_path + '/OPL_final.mat')
        OPL = dat["OPL_final"]

        # Create a 1x3 subplot grid with a smaller figure size
        fig, axs = plt.subplots(1, 3, figsize=(12, 4))
        fig.suptitle('Sample: {}'.format(folder_name))

        # Display the brightfield image in the first subplot
        im0 = axs[0].imshow(brightfield, aspect=0.5)
        axs[0].set_title('Brightfield Image')
        axs[0].set_xlabel('X-Axis')
        axs[0].set_ylabel('Y-Axis')
        axs[0].set_aspect(0.5)  # Adjust aspect ratio if needed
        axs[0].grid(False)
        plt.colorbar(im0, ax=axs[0])

        # Display the height map image in the second subplot
        im1 = axs[1].imshow(OPL, aspect=0.5)
        axs[1].set_title('Height Map (OPL)')
        axs[1].set_xlabel('X-Axis')
        axs[1].set_ylabel('Y-Axis')
        axs[1].set_aspect(0.5)  # Adjust aspect ratio if needed
        axs[1].grid(False)
        plt.colorbar(im1, ax=axs[1])

        # Plot the vertical trace of the heightmap in the third subplot

        axs[2].plot(np.linspace(0, 100, np.shape(OPL)[0]), OPL[:, section_x])
        axs[2].set_title('Vertical Trace of Heightmap at x={}'.format(section_x))
        axs[2].set_xlabel('Distance [$\mu m$]')
        axs[2].set_ylabel('Surface height [nm]')
        axs[2].grid(False)

        # Adjust spacing between subplots for better visualization
        fig.tight_layout()
        fig.subplots_adjust(top=0.88)

        return OPL
        
        # Show the plot
        plt.show()


        # fig.savefig('{}'.format(img_save_path + folder_name))

        
        
    except FileNotFoundError as e:
        # Handle the FileNotFoundError in a generic way
        print(f"Error processing folder '{folder_name}': {e}")

In [ ]:
folder_info = get_lowest_level_folders(root_dir)

for folder_name, folder_path in tqdm(folder_info):
    plot_images(folder_name=folder_name, folder_path=folder_path)

In [20]:
### 3D plotly surface plots

def surface3D_plots(folder_path, folder_name, img_save_path = '../Processed/8-30-23/'):
    
    try:
        # Load the height  image
        dat = loadmat(folder_path + '/OPL_final.mat')
        OPL = dat["OPL_final"]

        z_data= OPL 

        fig = go.Figure(data=[go.Surface(z=z_data, colorscale='rainbow')],
                    )

        fig.update_layout(title='3D surface plot', autosize=False,
                        template = 'plotly_dark',
                        scene=dict(aspectmode='cube')
                        )             
        fig.show()

        # this saves the image in a different orientation
        image_path = img_save_path + folder_name + '_3D.png'
        pio.write_image(fig, image_path, format='png', width=600, height=600, scale=2)

        name = 'eye = (x:0.1, y:0.1, z:1.5)'
        camera = dict(
            eye=dict(x=0.1, y=0.1, z=1.5)
        )

        fig.update_layout(scene_camera=camera, title=name)
        fig.show()

        # this saves the image in a different orientation
        image_path = img_save_path + folder_name + '_v2_3D.png'
        pio.write_image(fig, image_path, format='png', width=600, height=600, scale=2)

    except FileNotFoundError as e:
        # Handle the FileNotFoundError in a generic way
        print(f"Error processing folder '{folder_name}': {e}")

In [ ]:
# folder_info = get_lowest_level_folders(root_dir)

for folder_name, folder_path in tqdm(folder_info):
    surface3D_plots(folder_name=folder_name, folder_path=folder_path)